In [12]:
import csv
import json
import base64
import io
from pypdf import *
from bs4 import BeautifulSoup

from urllib.request import urlopen

key = "55228d9d4f216ad5972abd1be3c4c2e3"

In [3]:
def get_bill_ids(filePath):
    """Get bill ids from a csv file"""

    bill_ids = []
    with(open(filePath, 'r')) as f:
        reader = csv.reader(f)
        for row in reader:
            bill_ids.append(row[0])
    return bill_ids        

In [4]:
VA_bill_ids = get_bill_ids("VA/2024-2024_Regular_Session/csv/bills.csv")
PA_bill_ids = get_bill_ids("PA/2023-2024_Regular_Session/csv/bills.csv")
MN_bill_ids = get_bill_ids("MN/2023-2024_93rd_Legislature/csv/bills.csv")
CA_bill_ids = get_bill_ids("CA/2023-2024_Regular_Session/csv/bills.csv")

In [5]:
VA_bill_ids[:10]

['bill_id',
 '1785700',
 '1785447',
 '1785578',
 '1785663',
 '1785638',
 '1785434',
 '1785734',
 '1785841',
 '1785996']

In [6]:
def get_text_from_bill_id(bill_id):
    """API pull to get text from bill id
    returns None if bill id is not found
    returns empy string if no text is unavailable
    returns encoded text (with base64) if available
    """
    
    urlGetBill = f"https://api.legiscan.com/?key={key}&op=getBill&id={bill_id}"
    r1 = urlopen(urlGetBill).read().decode()
    data1 = json.loads(r1)
    if data1["status"] == "ERROR":
        return None
    if data1["bill"]["texts"] == []:
        return ""
    else:
        doc_id = data1["bill"]["texts"][0]["doc_id"]
        urlGetText = f"https://api.legiscan.com/?key={key}&op=getBillText&id={doc_id}"
        r2 = urlopen(urlGetText).read().decode()
        data2 = json.loads(r2)
        textEncoded = data2["text"]["doc"]
        return textEncoded

In [23]:
PA_bill_ids[1]

'1724272'

In [ ]:
# HTML text to plain text https://stackoverflow.com/questions/328356/extracting-text-from-html-file-using-python
data = get_text_from_bill_id(VA_bill_ids[2])
decoded = io.BytesIO(base64.b64decode(data))
soup = BeautifulSoup(decoded, 'html.parser')
for script in soup(["script", "style"]):
    script.extract()

text = soup.get_text()
lines = (line.strip() for line in text.splitlines())
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
text = '\n'.join(chunk for chunk in chunks if chunk)
type(text)
with open("HTML2TextTest.txt", "w") as f:
    f.write(text)

 def decodeHTMLTextToPlainText(data):
    decoded = io.BytesIO(base64.b64decode(data))
    soup = BeautifulSoup(decoded, 'html.parser')
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text   

In [ ]:
data = get_text_from_bill_id(PA_bill_ids[1])
decoded = base64.b64decode(data)
pdfBuffer = io.BytesIO(decoded)
reader = PdfReader(pdfBuffer)
print(len(reader.pages))
with open("PAPDF2Text.txt", "w") as f:
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        f.write(page.extract_text())

def decodePDFTextToPlainText(data):
    decoded = base64.b64decode(data)
    pdfBuffer = io.BytesIO(decoded)
    reader = PdfReader(pdfBuffer)
    text = ""
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        text += page.extract_text()
    return text

2


In [8]:
#Create a CSV file with the bill_id and the encoded text
bill_texts = []
bill_texts.append(["state","bill_id", "text"])
count = 0
for bill_id in VA_bill_ids: #skip the first line since its the header
    if bill_id == "bill_id":
        continue
    billText = get_text_from_bill_id(bill_id)
    count += 1
    if count % 100 == 0:
        print(count)
    bill_texts.append(["VA", bill_id, billText]) #If empty string then text was unavailable on LegiScan

# for bill_id in PA_bill_ids: #skip the first line since its the header
#     if bill_id == "bill_id":
#         continue
#     billText = get_text_from_bill_id(bill_id)
#     bill_texts.append(["PA", bill_id, billText]) #If empty string then text was unavailable on LegiScan
    
with open("bill_texts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(bill_texts)
        


100
200
